
### Step 1: Connect to ADLS

Since we are using Azure Credentials Passthrough connection method, we just need to use the custom access token and mount the folder that has all our TestStand files. 

In [0]:
# Install fsspec if needed
# %pip install fsspec

# Declare script variables
database_name = "dbo"

bronze_source = "abfss://bronze@arcdatalakeg2.dfs.core.windows.net/"
bronze_target = bronze_source + database_name
bronze_mnt = "/mnt/bronze/" + database_name

silver_source = "abfss://silver@arcdatalakeg2.dfs.core.windows.net/"
silver_target = silver_source + database_name
silver_mnt = "/mnt/silver/" + database_name

gold_source = "abfss://gold@arcdatalakeg2.dfs.core.windows.net/"
gold_target = gold_source + database_name
gold_mnt = "/mnt/gold/" + database_name

# Azure Credentials Passthrough configuration.
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}



### Step 2: Mount bronze container


In [0]:
# Add bronze_mnt if not already mounted.
if any(mount.mountPoint == bronze_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=bronze_source, mount_point=bronze_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(bronze_target)

[FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_ANALOGWAVEFORM.parquet', name='PROP_ANALOGWAVEFORM.parquet', size=343, modificationTime=1708270317000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_BINARY.parquet', name='PROP_BINARY.parquet', size=1044738, modificationTime=1708270299000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_DIGITALWAVEFORM.parquet', name='PROP_DIGITALWAVEFORM.parquet', size=311, modificationTime=1708270284000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_NUMERICLIMIT.parquet', name='PROP_NUMERICLIMIT.parquet', size=1289983, modificationTime=1708270268000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_RESULT.parquet', name='PROP_RESULT.parquet', size=23944627, modificationTime=1708270257000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/STEP_RESULT.parquet', name='STEP_RESULT.parquet', size=1


### Step 3: Mount silver container


In [0]:
# Add silver_mnt if not already mounted.
if any(mount.mountPoint == silver_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=silver_source, mount_point=silver_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(silver_target)

[]


### Step 4: Mount bronze container


In [0]:
# Add gold_mnt if not already mounted.
if any(mount.mountPoint == gold_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
      source=gold_source, mount_point=gold_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(gold_target)

[]


### Step 5: Setup Database and Extract Data to Tables in Databricks

We instantiate a DBFS database and USE it. Then we iterate over the TSDB directory at that level and create a table for each CSV file in that mount point. 


In [0]:
# Create and use DATABASE [drop IF ALREADY EXISTS]
spark.sql(f"drop database IF EXISTS {database_name} cascade ")
spark.sql(f"create database {database_name}")
spark.sql(f"use {database_name}")

# Create all tables into DBFS database
tables_list = dbutils.fs.ls(bronze_target)

for table in tables_list:
    table_name = table.name.strip(".parquet")
    spark.sql(f"DROP TABLE IF EXISTS `{table_name}`")
    spark.sql(f"""
              CREATE TABLE `{table_name}` 
              USING parquet 
              OPTIONS(path '{table.path}', header 'true', inferschema 'true')
              """)
    
# Display table
if debug_mode:
    df = spark.sql('select * from uut_result')
    display(df)

ID,STATION_ID,BATCH_SERIAL_NUMBER,TEST_SOCKET_INDEX,UUT_SERIAL_NUMBER,USER_LOGIN_NAME,START_DATE_TIME,EXECUTION_TIME,UUT_STATUS,UUT_ERROR_CODE,UUT_ERROR_MESSAGE,PART_NUMBER,TSR_FILE_NAME,TSR_FILE_ID,TSR_FILE_CLOSED,SSMA_TimeStamp
1,BRM-DQ1VH63,,-1,352Z9999,wgraba,2021-11-10T14:54:05Z,85.4034475,Error,5002,"[{ ""status"": true, ""code"": 5002, ""source"": ""Init DeviceNet Host.vi\n\nComplete call chain:\r\n Init DeviceNet Host.vi\r\n Init DeviceNet Host.vi.ProxyCaller"" },{ ""status"": true, ""code"": 1172, ""source""",,null,null,false,AAAAAAAf3jc=
2,BRM-DQ1VH63,,-1,352C0000,mfgtech,2021-11-10T15:00:30Z,545.1689131,Passed,0,,,null,null,false,AAAAAAAf3jg=
3,BRM-DQ1VH63,,-1,352C3129,mfgtech,2021-11-11T03:55:27Z,538.7750436,Failed,0,,,null,null,false,AAAAAAAf3jk=
4,BRM-DQ1VH63,,-1,352C3129,mfgtech,2021-11-11T04:04:53Z,534.2498016,Passed,0,,,null,null,false,AAAAAAAf3jo=
5,BRM-DQ1VH63,,-1,352C3134,mfgtech,2021-11-11T04:16:34Z,551.1659191,Passed,0,,,null,null,false,AAAAAAAf3js=
6,BRM-DQ1VH63,,-1,352C3136,mfgtech,2021-11-11T04:26:20Z,156.0671306,Error,-17300,Parameter 'Instrument Refs': The Object Reference is set to Nothing.,,null,null,false,AAAAAAAf3jw=
7,BRM-DQ1VH63,,-1,352C3132,mfgtech,2021-11-11T04:30:07Z,23.6520898,Terminated,0,,,null,null,false,AAAAAAAf3j0=
8,BRM-DQ1VH63,,-1,352C3132,mfgtech,2021-11-11T04:41:02Z,525.8419235,Passed,0,,,null,null,false,AAAAAAAf3j4=
9,BRM-DQ1VH63,,-1,352C3125,mfgtech,2021-11-11T05:06:29Z,656.8235001,Passed,0,,,null,null,false,AAAAAAAf3j8=
10,BRM-DQ1VH63,,-1,352C3125,mfgtech,2021-11-11T05:17:39Z,666.4288131,Passed,0,,,null,null,false,AAAAAAAf3kA=
